In [ ]:
!pip3 install ktrain

     |████████████████████████████████| 25.2MB 128kB/s 
     |████████████████████████████████| 421.8MB 23kB/s 
     |████████████████████████████████| 6.7MB 45kB/s 
     |████████████████████████████████| 983kB 36.6MB/s 
     |████████████████████████████████| 245kB 41.4MB/s 
     |████████████████████████████████| 757kB 31.7MB/s 
     |████████████████████████████████| 471kB 34.9MB/s 
     |████████████████████████████████| 450kB 39.8MB/s 
     |████████████████████████████████| 3.9MB 33.3MB/s 
     |████████████████████████████████| 3.0MB 33.3MB/s 
     |████████████████████████████████| 890kB 34.8MB/s 
     |████████████████████████████████| 1.1MB 33.4MB/s 
  Created wheel for ktrain: filename=ktrain-0.17.3-cp36-none-any.whl size=25249269 sha256=8551ee8fc76ed06a82219e6239dc1e650e282a1089fb0117b52bfa9c4511db56
  Stored in directory: /root/.cache/pip/wheels/24/87/78/8dcbc028acc22bc106e03a954f0de2f02e57fb42438ecb177d
  Created wheel for keras-bert: filename=keras_bert-0.84.0-cp36-none

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

import ktrain
from ktrain import text

In [ ]:
import nltk
import bs4
import re
#nltk.download('stopwords')
nltk.download('punkt')
#from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize
#stop_words = set(stopwords.words('english'))
def clean_tweet(tweet):
  tweet=bs4.BeautifulSoup(tweet,'lxml').get_text()
  tweet=re.sub(r'@[A-Za-z0-9]+',' ',tweet)
  tweet=re.sub(r'https?://[A-Za-z0-9./]+',' ',tweet)
  tweet = re.sub(r'&amp;', '&', tweet)
  #tweet=re.sub(r"[^a-zA-Z']",' ',tweet)
  tweet=re.sub(r" +"," ",tweet)
  tweet=tweet.strip()
  #tweet=tweet.lower()
  #word_tokens = word_tokenize(tweet)
  #filtered_tweet = [w for w in word_tokens if not w in stop_words]
  #return ' '.join(filtered_tweet)
  return tweet

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
data_train = pd.read_csv('/content/drive/My Drive/IBM_sentiment_analysis/classification_sentiment_analysis/datasets/self-training_iteration1_data/train_f_n.csv', encoding='utf-8')
data_test = pd.read_csv('/content/drive/My Drive/IBM_sentiment_analysis/classification_sentiment_analysis/datasets/self-training_iteration1_data/test_f_n.csv', encoding='utf-8')

In [ ]:
print(data_train.shape)
print(data_test.shape)
print(data_train.isnull().sum())
print(data_test.isnull().sum())

(11231, 2)
(3744, 2)
Emotion    0
Text       0
dtype: int64
Emotion    0
Text       0
dtype: int64


In [ ]:
data_train.head()

,Emotion,Text
0,anger,When I heard that I was not to be provided wit...
1,sadness,@GoPro your UX online is appalling! No clear s...
2,sadness,We'll get to go fishing at the river . I hate...
3,joy,"Hi , Mary , long time no see ."
4,anger,To my great annoyance I found myself jealous .


In [ ]:
data_test.head()

,Emotion,Text
0,neutral,Next june .
1,fear,Awesome time at @teamoneill 's Fighting Fear p...
2,anger,@o2academybham since when the fuck can you not...
3,anger,Losing the will 2 live with @virginmedia busin...
4,anger,A whole crowd of us went to a folklore festiva...


In [ ]:
data_train['cleaned']=data_train['Text'].apply(clean_tweet)
data_test['cleaned']=data_test['Text'].apply(clean_tweet)

In [ ]:
data_train.head()

,Emotion,Text,cleaned
0,anger,When I heard that I was not to be provided wit...,When I heard that I was not to be provided wit...
1,sadness,@GoPro your UX online is appalling! No clear s...,your UX online is appalling! No clear save but...
2,sadness,We'll get to go fishing at the river . I hate...,We'll get to go fishing at the river . I hate ...
3,joy,"Hi , Mary , long time no see .","Hi , Mary , long time no see ."
4,anger,To my great annoyance I found myself jealous .,To my great annoyance I found myself jealous .


In [ ]:
data_test.head()

,Emotion,Text,cleaned
0,neutral,Next june .,Next june .
1,fear,Awesome time at @teamoneill 's Fighting Fear p...,Awesome time at 's Fighting Fear premiere last...
2,anger,@o2academybham since when the fuck can you not...,since when the fuck can you not stand at a con...
3,anger,Losing the will 2 live with @virginmedia busin...,Losing the will 2 live with business bb gone d...
4,anger,A whole crowd of us went to a folklore festiva...,A whole crowd of us went to a folklore festiva...


In [ ]:
data_train=data_train[data_train['cleaned']!='']
data_test=data_test[data_test['cleaned']!='']

In [ ]:
print(data_train.shape)
print(data_test.shape)
print(data_train.isnull().sum())
print(data_test.isnull().sum())

(11231, 3)
(3744, 3)
Emotion    0
Text       0
cleaned    0
dtype: int64
Emotion    0
Text       0
cleaned    0
dtype: int64


In [ ]:
X_train = data_train.cleaned.tolist()
X_test = data_test.cleaned.tolist()

y_train = data_train.Emotion.tolist()
y_test = data_test.Emotion.tolist()

data = data_train.append(data_test, ignore_index=True)

class_names = ['joy', 'sadness', 'fear', 'anger', 'neutral']

print('size of training set: %s' % (len(data_train['Text'])))
print('size of validation set: %s' % (len(data_test['Text'])))
print(data.Emotion.value_counts())

data.head(10)

size of training set: 11231
size of validation set: 3744
fear       3148
joy        3049
anger      2952
sadness    2914
neutral    2912
Name: Emotion, dtype: int64


,Emotion,Text,cleaned
0,anger,When I heard that I was not to be provided wit...,When I heard that I was not to be provided wit...
1,sadness,@GoPro your UX online is appalling! No clear s...,your UX online is appalling! No clear save but...
2,sadness,We'll get to go fishing at the river . I hate...,We'll get to go fishing at the river . I hate ...
3,joy,"Hi , Mary , long time no see .","Hi , Mary , long time no see ."
4,anger,To my great annoyance I found myself jealous .,To my great annoyance I found myself jealous .
5,anger,I had too much homework and examinations and I...,I had too much homework and examinations and I...
6,sadness,You know that my father is irritable .,You know that my father is irritable .
7,joy,It was the first time I met him (my boyfriend ...,It was the first time I met him (my boyfriend ...
8,neutral,The weather report says it's going to rain fo...,The weather report says it's going to rain for...
9,sadness,When my basket ball team lost the qualificatio...,When my basket ball team lost the qualificatio...


In [ ]:
encoding = {
    'joy': 0,
    'sadness': 1,
    'fear': 2,
    'anger': 3,
    'neutral': 4
}


y_train = [encoding[x] for x in y_train]
y_test = [encoding[x] for x in y_test]

In [ ]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=X_train, y_train=y_train,
                                                                       x_test=X_test, y_test=y_test,
                                                                       class_names=class_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=35000)

task: text classification
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [ ]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)

Is Multi-Label? False
maxlen is 350
done.


In [ ]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train), 
                             val_data=(x_test, y_test),
                             batch_size=6)

In [ ]:
learner.fit_onecycle(2e-5, 3)



begin training using onecycle policy with max lr of 2e-05...
Train on 11231 samples, validate on 3744 samples
Epoch 1/3
11231/11231 [==============================] - 2325s 207ms/sample - loss: 0.8235 - accuracy: 0.6877 - val_loss: 0.5426 - val_accuracy: 0.8080
Epoch 2/3
11231/11231 [==============================] - 2327s 207ms/sample - loss: 0.3967 - accuracy: 0.8622 - val_loss: 0.4699 - val_accuracy: 0.8285
Epoch 3/3
11231/11231 [==============================] - 2325s 207ms/sample - loss: 0.1554 - accuracy: 0.9484 - val_loss: 0.5116 - val_accuracy: 0.8392


In [ ]:
learner.validate(val_data=(x_test, y_test), class_names=class_names)

              precision    recall  f1-score   support

         joy       0.87      0.89      0.88       762
     sadness       0.80      0.83      0.82       729
        fear       0.86      0.81      0.84       787
       anger       0.81      0.83      0.82       738
     neutral       0.85      0.83      0.84       728

    accuracy                           0.84      3744
   macro avg       0.84      0.84      0.84      3744
weighted avg       0.84      0.84      0.84      3744



array([[676,  17,  13,   9,  47],
       [ 20, 603,  32,  60,  14],
       [ 27,  53, 641,  47,  19],
       [  7,  48,  40, 615,  28],
       [ 48,  29,  17,  27, 607]])

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.get_classes()

['joy', 'sadness', 'fear', 'anger', 'neutral']

In [ ]:
message = 'hell no'
predictor.predict(message)

'anger'

In [ ]:
predictor.save("/content/drive/My Drive/IBM_sentiment_analysis/classification_sentiment_analysis/models/bert_model_self-training_iteration1")